<a href="https://colab.research.google.com/github/GeorgeGuevara/Actividad1/blob/master/Actividad1_distribuido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
inputfile = open('/content/drive/MyDrive/paginas.txt',"r")
text = inputfile.read()

***Extraer texto de la web***

In [14]:
import requests   # Importa la librería requests para realizar solicitudes HTTP
from bs4 import BeautifulSoup   # Importa la clase BeautifulSoup de la librería BeautifulSoup para analizar el HTML

def extract_text_from_web_pages(text):
    urls = text.split()   # Divide el texto de entrada en URLs separadas
    extracted_text = ""   # Inicializa una cadena vacía para almacenar el texto extraído de las páginas web

    for url in urls:   # Itera sobre cada URL en la lista de URLs
        response = requests.get(url)   # Realiza una solicitud HTTP GET a la URL
        if response.status_code == 200:   # Verifica si la solicitud fue exitosa (código de estado 200)
            soup = BeautifulSoup(response.text, 'html.parser')   # Crea un objeto BeautifulSoup para analizar el contenido HTML de la página
            text = ' '.join([p.get_text() for p in soup.find_all('p')])   # Extrae el texto de todos los elementos de párrafo ('p') en la página
            extracted_text += text + "\n"   # Agrega el texto extraído a la cadena de texto final con un salto de línea

    return extracted_text   # Devuelve el texto extraído de todas las páginas web

***Data Cleaning Function***

In [15]:
def data_clean(text):
  NoNumbers = ''.join([i for i in text if not i.isdigit()]) #Eliminando números
  NoNumbers = text.lower()                                  #Hacer el texto en minúsculas
  import re
  onlyText = re.sub(r"[^a-z\s]+",' ',NoNumbers)             #Eliminando puntuación
  finaltext = "".join([s for s in onlyText.strip().splitlines(True) if s.strip()]) #Eliminando las líneas nulas
  return finaltext

***Division (Split) Function***

In [16]:
def splitlines(text, a):
    linessplit = text.splitlines()   # Divide el texto en líneas y lo almacena en una lista
    split1 = linessplit[0:a]   # Crea la primera división con las primeras "a" líneas en la división 1
    split2 = linessplit[a:]   # Crea la segunda división con las líneas restantes a partir de la línea "a" en la división 2
    return split1, split2   # Devuelve las dos divisiones como tuplas

**Función de asignador (Mapper)***

In [17]:
def mapper(text, out_queue):
    keyval = []   # Inicializa una lista para almacenar pares clave-valor resultantes del mapeo

    # Itera sobre cada línea de texto recibida como entrada
    for i in text:
        wordssplit = i.split()   # Divide la línea en palabras individuales
        for j in wordssplit:
            keyval.append([j, 1])   # Agrega cada palabra de la línea con el valor 1 en el formato ["word", 1] a la lista de pares clave-valor

    out_queue.put(keyval)   # Coloca la lista de pares clave-valor en la cola de salida especificada

***Función de clasificación***

In [18]:
def sortedlists(list1, list2):
    out = list1 + list2   # Concatena las dos listas de entrada en una sola lista

    out.sort(key=lambda x: x[0]) # Ordena la lista combinada basándose en el primer elemento de cada sublista, que es la "palabra"


    return out   # Devuelve la lista combinada y ordenadadef sortedlists(list1, list2):


***Partición Funsión***

In [19]:
def partition(sorted_list):
    sort1out = []   # Inicializa una lista para almacenar palabras que comienzan con 'a-m'
    sort2out = []   # Inicializa una lista para almacenar palabras que comienzan con 'n-z'

    # Itera sobre cada elemento de la lista de palabras ordenadas
    for i in sorted_list:
        if i[0][0] < 'n':   # Comprueba si la primera letra de la palabra está antes de 'n' en el orden alfabético
            sort1out.append(i)   # Agrega la palabra a la lista sort1out si comienza con 'a-m'
        else:
            sort2out.append(i)   # Agrega la palabra a la lista sort2out si comienza con 'n-z'

    return sort1out, sort2out   # Devuelve dos listas separadas, una con palabras de 'a-m' y otra con palabras de 'n-z'

***Reducer Function***

In [20]:
def reducer(part_out1, out_queue):
    sum_reduced = []   # Inicializa una lista para almacenar las palabras reducidas con su recuento

    count = 1   # Inicializa el contador de palabras a 1

    for i in range(0, len(part_out1)):
        if i < len(part_out1) - 1:
            if part_out1[i] == part_out1[i + 1]:
                count = count + 1   # Incrementa el contador si la palabra actual es igual a la siguiente palabra
            else:
                sum_reduced.append([part_out1[i][0], count])   # Agrega la palabra junto con su recuento a la lista sum_reduced en el formato ["palabra", recuento]
                count = 1   # Restablece el contador a 1
        else:
            sum_reduced.append(part_out1[i])   # Agrega la última palabra a la lista de salida

    out_queue.put(sum_reduced)   # Coloca la lista de palabras reducidas y sus recuentos en la cola de salida

***función Multi - Threading :***
la función definida por el usuario a continuación toma una función y dos entradas como argumentos. La función se aplica en ambas entradas simultáneamente y la función devuelve la salida.

In [21]:
import threading   # Importa la biblioteca threading para la programación multihilo
import queue   # Importa la clase Queue de la biblioteca queue para la comunicación entre subprocesos

def multi_thread_function(func, map1_input, map2_input):
    my_queue1 = queue.Queue()   # Crea una cola para almacenar los valores de salida del mapeador 1
    my_queue2 = queue.Queue()   # Crea una cola para almacenar los valores de salida del mapeador 2

    # Crea dos subprocesos que ejecutarán la función 'func' con las entradas 'map1_input' y 'map2_input'
    t1 = threading.Thread(target=func, args=(map1_input, my_queue1))
    t2 = threading.Thread(target=func, args=(map2_input, my_queue2))

    t1.start()   # Inicia la ejecución del hilo 1
    t2.start()   # Inicia la ejecución del hilo 2 para ejecutarse simultáneamente con el hilo 1

    t1.join()   # Espera a que el hilo 1 se ejecute por completo
    t2.join()   # Espera a que el hilo 2 se ejecute por completo

    list1out = my_queue1.get()   # Obtiene los valores de la cola del hilo 1 en una variable
    list2out = my_queue2.get()   # Obtiene los valores de la cola del hilo 2 en una variable

    return list1out, list2out   # Devuelve los valores obtenidos de los hilos 1 y 2

***Función Main***

In [22]:
def main_function(text):
  textweb = extract_text_from_web_pages(text)
  cleantext = data_clean(textweb)
  linessplit = splitlines(cleantext,5000)
  mapperout = multi_thread_function(mapper,linessplit[0],linessplit[1])
  sortedwords = sortedlists(mapperout[0],mapperout[1])
  slicedwords = partition(sortedwords)
  reducerout = multi_thread_function(reducer,slicedwords[0],slicedwords[1])
  return reducerout[0]+reducerout[1]

output = main_function(text)
import pandas as pd
pd.DataFrame(output).to_csv("Output.csv",index=False,header = ["Word","Frequency"]) #Saving file as a .csv file in the current directory